**Nono Trabalho**

**Nome: Valéria Cristina A. R. de Figueredo**

In [42]:
! pip install langchain langchain_openai langchain_community

In [43]:
! pip install langchain-openai

In [44]:
! pip install faiss-cpu

In [45]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [46]:
#https://python.langchain.com/v0.2/docs/how_to/ #document-loaders
#https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

In [47]:
print(docs[0].page_content)

%HEADER_COMPANY_WEBSITE%%HEADER_EMPLOYEES%%BREADCRUMB_JOB_OPENINGS%Pesquisador em Inteligência Artificial e Sistemas Distribuídos%BUTTON_APPLY_TO_POSITION%%BUTTON_APPLY_USING_INDEED%%BUTTON_APPLY_USING_LINKED_IN%Olá, nós somos o CESAR! 
Somos um centro de inovação e educação que, há quase 30 anos, forma pessoas e impulsiona organizações, potencializando suas estratégias digitais. Nosso foco está em resolver problemas complexos e desafiadores em um ambiente de trabalho descontraído e descentralizado, com inúmeros benefícios para nossos colaboradores. Aqui, você será protagonista, interagindo diretamente com clientes de escala global.
Como Pesquisador em Inteligência Artificial e Sistemas Distribuídos, sua missão será conduzir pesquisas inovadoras, integrando IA e sistemas distribuídos com foco em cibersegurança. Você vai transformar pesquisas em soluções escaláveis, colaborar com equipes multidisciplinares e compartilhar boas práticas para impulsionar avanços tecnológicos. Está pronto p

### Dividindo Documentos - Splitting/Chunking

In [48]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)


6

In [49]:
all_splits

[Document(metadata={'source': 'https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos', 'start_index': 0}, page_content='%HEADER_COMPANY_WEBSITE%%HEADER_EMPLOYEES%%BREADCRUMB_JOB_OPENINGS%Pesquisador em Inteligência Artificial e Sistemas Distribuídos%BUTTON_APPLY_TO_POSITION%%BUTTON_APPLY_USING_INDEED%%BUTTON_APPLY_USING_LINKED_IN%Olá, nós somos o CESAR! \nSomos um centro de inovação e educação que, há quase 30 anos, forma pessoas e impulsiona organizações, potencializando suas estratégias digitais. Nosso foco está em resolver problemas complexos e desafiadores em um ambiente de trabalho descontraído e descentralizado, com inúmeros benefícios para nossos colaboradores. Aqui, você será protagonista, interagindo diretamente com clientes de escala global.'),
 Document(metadata={'source': 'https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos', 'start_index': 237}, page_content='Somos um centro d

In [50]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [51]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
embeddings_model = OpenAIEmbeddings()


In [52]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())


In [53]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [54]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [55]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [56]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [57]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [59]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como parte de seu pacote de benefícios. Além disso, também disponibiliza plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [74]:
# 1- Criação de um arquivo de texto com informações fictícias, que a LLM não conhece
text_content = """
O valor do salário mínimo de Lima em 2025 será de $ 2.320,00.
A capital do Limoeiro é Liminha, conhecida por ser o maior centro financeiro da América Latina.
O Sport Clube de Lima é um dos clubes de futebol mais populares do Limoeiro, com sede na cidade de Lima.
O Rio LimaJuice é o maior rio do Limoeiro em volume de água e percorre parte das cidades de Liminha, Laranja e Maracuja.
"""

# Salvar o conteúdo em um arquivo de texto
with open("informacoes.txt", "w") as f:
    f.write(text_content)


from langchain.document_loaders import TextLoader

# 2- Carregar o arquivo de texto
loader = TextLoader("informacoes.txt")
docs = loader.load()

# Configurar o text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # Defina o tamanho do chunk
    chunk_overlap=50,  # Sobreposição entre os chunks
    add_start_index=True
)

# Dividir os documentos
all_splits = text_splitter.split_documents(docs)

from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 3- Criar o vetor de embeddings a partir dos documentos
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

# 4 - Criar o retriever a partir do vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

from langchain.prompts import ChatPromptTemplate

# 5- Definir o template do prompt
system_template = """
Você é um assistente com tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta.
Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

#gerando respostas
from langchain_openai import ChatOpenAI

# 6- Usar o modelo de linguagem GPT
llm = ChatOpenAI(model="gpt-4")

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)


for chunk in rag_chain.stream("Qual é a capital do Limoeiro?"):
    print(chunk, end="", flush=True)
#resposta
# A capital do Limoeiro é Liminha. (informação falsa)

[HumanMessage(content='\nVocê é um assistente com tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta.\nSe você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]
A capital do Limoeiro é Liminha.